In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier # 60
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier

In [ ]:
# !pip install lightgbm

In [ ]:
# !pip install xgboost

In [14]:
dog = pd.read_csv('TotalDog.csv', encoding='euc-kr')


## 전처리


In [15]:

dog['의료비용'] = dog['의료비용'].replace('35u','30u')
dog['의료비용'] = dog['의료비용'].replace('40u','30o')
dog['의료비용'] = dog['의료비용'].replace('35o','30o')
dog['이름'] = dog['이름'].replace('말티지','말티즈')
dog['이름'] = dog['이름'].replace('발바리','믹스견')
dog['이름'] = dog['이름'].replace('요키','믹스견')
dog['이름'] = dog['이름'].replace('포메리안','포메라니안')
dog['이름'].unique()
dog['이름'].value_counts()


dog['이름'] = dog[dog["이름"] !="슈나우저"]
dog['이름'] = dog[dog["이름"] !="비숑"]
dog['이름'] = dog[dog["이름"] !="시바견"]
dog['이름'] = dog[dog["이름"] !="삽살개"]
dog['이름'] = dog[dog["이름"] !="사모예드"]
dog['이름'] = dog[dog["이름"] !="아키타"]
dog['이름'] = dog[dog["이름"] !="그레이 하운드"]
dog['이름'] = dog[dog["이름"] !="도베르만"]
dog.dropna(inplace=True)

Title_Dictionary = {
     "말티즈" : 0,
    "믹스견":1,
    "푸들" : 2,
    "포메라니안" : 3,
    "진돗개": 4,
    "시츄":5,
    "허스키" : 6,
    "치와와":7,
    "미니핀": 8,
    "코카스파니엘": 9,
    "요크셔테리어" : 10,
     "웰시코기":11,
    "러프콜리": 12,
   "달마시안": 13,
    "골든 리트리버" : 14,
    "미니어쳐 핀셔" : 15,
      "파피용" : 16,
     "셰퍼트" :  17,
     "차우차우" : 18,
     "닥스훈트" : 19,
   "보더콜리" : 20,
     "말라뮤트" : 21,
    "불독" : 22,
   "풍산개" : 23  
}

dog['이름'] = dog['이름'].map(Title_Dictionary)

onehot_column = ['크기','거주환경','실내실외','의료비용']


for col in onehot_column:
    dummy = pd.get_dummies(dog[col],prefix = col)
    # 원핫인코딩된 결과물을 기존의 자료와 합치기
    dog = pd.concat([dog,dummy], axis = 1)
    # 글자형 컬럼(원핫인코딩 진행한) 삭제
    dog.drop(col,axis = 1, inplace = True)

dummy

,의료비용_10u,의료비용_30o,의료비용_30u
50,1,0,0
51,1,0,0
52,1,0,0
53,0,0,1
54,1,0,0
...,...,...,...
1709,1,0,0
1710,1,0,0
1711,0,1,0
1712,1,0,0


,이름,외로움,운동량,털빠짐,친화성(애교),개짖음,집지키기,크기_L,크기_M,크기_S,거주환경_A,거주환경_J,실내실외_I,실내실외_IO,실내실외_O,의료비용_10u,의료비용_30o,의료비용_30u
50,19,5,4,3,3,4,3,0,1,0,1,0,1,0,0,1,0,0
51,19,4,5,5,3,3,3,0,1,0,1,0,1,0,0,1,0,0
52,19,3,4,5,3,4,3,0,0,1,1,0,1,0,0,1,0,0
53,19,4,3,4,3,5,4,0,1,0,1,0,1,0,0,0,0,1
54,19,4,4,5,3,3,2,0,1,0,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,14,2,4,5,4,3,2,1,0,0,0,1,0,1,0,1,0,0
1710,14,1,3,5,5,2,2,0,1,0,0,1,0,1,0,1,0,0
1711,14,2,3,4,5,3,3,1,0,0,0,1,0,1,0,0,1,0
1712,14,1,4,5,5,2,3,0,1,0,0,1,0,1,0,1,0,0


# 모델 선택 및 하이퍼파라미터 튜닝
# 문제와 정답으로 분리 - X, y

In [17]:
y = dog.iloc[:,0]
X = dog.iloc[:,1:]
ans = dog.iloc[5,1:]


X_train, X_test, y_train, y_test = train_test_split(
                                X, #문제데이터
                                y,
                                test_size = 0.3, 
                                random_state = 3 
)


## 학습, 평가

In [19]:
test_list= []
train_list= []  # n_neighbors = 2
for i in range(1,101):
    
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train,y_train)
    
    train_list.append(model.score(X_train,y_train))

train_list

for i in range(1,101):
    
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train,y_train)
    
    test_list.append(model.score(X_test,y_test))

test_list

knn_model = KNeighborsClassifier(n_neighbors= 12,
                                 p=1,
                                 weights = 'distance'
                                )
knn_model

knn_model.fit(X_train,y_train)
knn_pred = knn_model.predict(X_test)
# print('XGB : %.2f'%(metrics.accuracy_score(knn_pred,y_test)*100))


# RandomForestClassifier


scaler = MinMaxScaler()
scaler.fit(X_train)
trans_X_train = scaler.transform(X_train)
trans_X_test = scaler.transform(X_test)
Rand = RandomForestClassifier()
Rand.fit(trans_X_train,y_train)
Rand_pred = Rand.predict(X_test)
# print('Random : %.2f'%(metrics.accuracy_score(Rand_pred,y_test)*100))

# LGBMClassifier


LGB = LGBMClassifier(max_depth= 5,
                    max_features =0.5,
                    max_leaf_nodes= 20,
                    min_samples_leaf= 10,
                    n_estimators= 1000)
LGB.fit(X_train,y_train)
LGB_pred = LGB.predict(X_test)
# LGB.score(X_test,y_test)
# print('LGB : %.2f'%(metrics.accuracy_score(LGB_pred,y_test)*100))

# XGBClassifier

XGB = XGBClassifier(max_depth= 5,
                    max_features= 0.5, 
                    max_leaf_nodes= 20,
                    min_samples_leaf= 10, 
                    n_estimators= 2000)
XGB.fit(X_train,y_train)
XGB_pred = XGB.predict(X_test)
# print('XGB : %.2f'%(metrics.accuracy_score(XGB_pred,y_test)*100))


# tree
tree = DecisionTreeClassifier()
result = cross_val_score(tree, X_train, y_train, cv = 5)
result.mean()
tree = DecisionTreeClassifier()
tree.fit(X_train,y_train)
tree.score(X_train,y_train)
tree.score(X_test,y_test)


# LinearSVC

svm = LinearSVC()
svm.fit(X_train,y_train)
svm_pred = svm.predict(X_test)
# print('SVM: %.2f'%(metrics.accuracy_score(svm_pred,y_test)*100))


 # DecisionTreeClassifier
dog_dtc = DecisionTreeClassifier()
dog_dtc.fit(X_train,y_train)
Desi_pred = dog_dtc.predict(X_test)
# print('Decision: %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))

# # KNeighborsClassifier
# dog_knn = KNeighborsClassifier(n_neighbors=3)
# dog_knn.fit(X_train,y_train)
# dog_pred = dog_knn.predict(X_test)
# # print('Knn: %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))

# LogisticRegression
dog_lrg = LogisticRegression()
dog_lrg.fit(X_train,y_train)
dog_pred = dog_lrg.predict(X_test)
#print(dog_pred)
print('Logistic : %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))

# dog_rfc = RandomForestClassifier()
# dog_rfc.fit(X_train,y_train)
# dog_pred = dog_rfc.predict(X_test)
# # print('RandomForest : %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))

forest = RandomForestClassifier(n_estimators=1000,
                                # tree의 개수
                                      max_features=0.7,# 각각의 트리에서 사용할 특성의 개수
                                      max_depth=5,
                                      min_samples_leaf=15,
                                      max_leaf_nodes=50)

forest.fit(X_train,y_train)
forest_pred = forest.predict(X_test)
# forest.score(X_test,y_test)


# 개발모델
knn_model = KNeighborsClassifier(n_neighbors= 12,
                                 p=1,
                                 weights = 'distance'
                                )
# XGB = XGBClassifier()
LGB = LGBMClassifier()
Rand = RandomForestClassifier(max_depth=3)

#최종모델
# Rand = RandomForestClassifier()
XGB = XGBClassifier()

knn_model.fit(X_train,y_train)
# XGB.fit(X_train,y_train)
LGB.fit(X_train,y_train)
Rand.fit(X_train,y_train)

knn_pred = knn_model.predict(X_test)
# xgb_pred = XGB.predict(X_test)
lgb_pred = LGB.predict(X_test)
rand_pred = Rand.predict(X_test)
# lgb_pred

new_data = np.array([knn_pred,rand_pred,lgb_pred])
# new_data.shape

new_data = np.transpose(new_data)
# new_data.shape

# new_data[:10]

# 스태킹 최종 결과

# Rand.fit(new_data,y_test)
# Rand_pred = Rand.predict(new_data)
XGB.fit(new_data,y_test)
xgb_pred = XGB.predict(new_data)

# print("Rand : {0:.4f}".format(accuracy_score(Rand,Rand_pred)))


print('Random : %.2f'%(metrics.accuracy_score(Rand_pred,y_test)*100))
print('knn : %.2f'%(metrics.accuracy_score(knn_pred,y_test)*100))
print('LGB : %.2f'%(metrics.accuracy_score(LGB_pred,y_test)*100))
print('XGB : %.2f'%(metrics.accuracy_score(XGB_pred,y_test)*100))
print('SVM: %.2f'%(metrics.accuracy_score(svm_pred,y_test)*100))
print('Decision: %.2f'%(metrics.accuracy_score(Desi_pred,y_test)*100))
print('forest: %.2f'%(metrics.accuracy_score(forest_pred,y_test)*100))
print('Logistic : %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))
print('RandomForest : %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))
print('스태킹 XGB : %.2f'%(metrics.accuracy_score(xgb_pred,y_test)*100))



[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: max_leaf_nodes
[LightGBM] [Warning] Unknown parameter: min_samples_leaf


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:48:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "max_features", "max_leaf_nodes", "min_samples_leaf" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:48:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic : 62.01


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:48:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random : 14.78
knn : 62.53
LGB : 66.75
XGB : 65.17
SVM: 59.89
Decision: 54.88
forest: 52.77
Logistic : 62.01
RandomForest : 62.01
스태킹 XGB : 86.81


array([[2.1329039e-04, 2.2217559e-04, 1.4020606e-04, 2.8013188e-04,
        8.6261556e-05, 3.0769344e-04, 3.1580889e-04, 1.6281336e-04,
        2.1872878e-04, 3.2161444e-04, 1.3223139e-03, 3.0455334e-04,
        4.0805424e-04, 1.5044863e-03, 9.9025553e-01, 3.8960841e-04,
        3.7480923e-04, 2.3043172e-04, 2.6647447e-04, 7.8102385e-05,
        7.3722121e-04, 1.8416614e-04, 1.6142036e-04, 1.5140552e-03]],
      dtype=float32)

1순위 :  0.99025553
2순위 :  0.0015140552
3순위 :  0.0015044863


골든리트리버
풍산개
달마시안


In [ ]:
from flask import Flask
from flask import request, redirect

app = Flask(__name__) 

@app.route('/dogmc', methods=['GET'])
def index(): 
#정답 받아오기
    v1 = request.args['v1']
    v2 = request.args['v2']
    v3 = request.args['v3']
    v4 = request.args['v4']
    v5 = request.args['v5']
    v6 = request.args['v6']
    v7 = request.args['v7']
    v8 = request.args['v8']
    v10 = request.args['v10']
    v11 = request.args['v11']

    predict_result = dog.iloc[10,1:]
    #정답지 백지화
    for i in range(len(predict_result)):
        predict_result[i] = 0
    #정답 입력
    if v1 == 'S':
        predict_result[9] =1
    elif v1 == 'M':
        predict_result[8] =1
    elif v1 == 'L':
        predict_result[7]= 1
    if v2 == 'A':
        predict_result[10] = 1
    elif v1 == 'J':
        predict_result[11] = 1
    if v3 == 'I':
        predict_result[12] = 1
    elif v3 == 'O':
        predict_result[14] = 1
    elif v3 == 'IO':
        predict_result[13] = 1

    predict_result[2]=v5
    predict_result[3]=v6
    predict_result[4]=v7

    if v8 == '10u':
        predict_result[15] = 1
    elif v8 == '30o':
        predict_result[16] = 1
    elif v8 == '30u':
        predict_result[17] = 1
        
    predict_result[5] = v10
    predict_result[6] = v11
    
    #차원 맞추기
    np_result = np.array(predict_result)
    np_result = np_result.reshape(1,-1)
    #기반모델 사용 예측
    knn_dogresult = knn_model.predict(np_result)
    rand_dogresult = Rand.predict(np_result)
    lgb_dogresult = LGB.predict(np_result)
    #기반모델정답 합치기
    new_dogresult = np.array([knn_dogresult,rand_dogresult,lgb_dogresult])
    #transpose로 피쳐를 axis = 1로 만들기
    new_dogresult = np.transpose(new_dogresult)

    #최종모델사용
    pred= XGB.predict_proba(new_dogresult)

    # 순위 찾기
    # 1순위
    rank1 = np.max(pred)
    print("1순위 : " , rank1)
    # 2순위
    temp_list1 = []
    temp = pred[0][0]
    for i in range(1,pred.size):
        if pred[0][i] < np.max(pred):
            temp_list1.append(pred[0][i])
    rank2 = max(temp_list1)
    print("2순위 : " , rank2)
    # 3순위
    temp_list3 = []
    for i in range(1,pred.size):
        if pred[0][i] < rank2:
            temp_list3.append(pred[0][i])
    rank3 = max(temp_list3)
    print("3순위 : ",rank3)
    #리스트에 순위담아주기
    list = [rank1,rank2,rank3]
    #결과 리스트 생성
    result_list = [0,0,0]
    #결과를 견종리스트 인덱스번호에 맞게 담아주기
    for i in range(0,3,1):
        for j in range(pred.size):
            if list[i] == pred[0][j]:
                result_list[i]= j
    #최종적으로 이클립스에 보낼 리스트생성
    send_list = [0,0,0]
    #견종리스트
    dog_list = ['말티즈',
     '믹스견',
     '푸들',
     '포메라니안',
     '진돗개',
     '시츄',
     '허스키',
     '치와와',
     '미니핀',
     '코카스파니엘',
     '요크셔테리어',
     '웰시코기',
     '러프콜리',
     '달마시안',
     '골든 리트리버',
     '미니어쳐 핀셔',
     '파피용',
     '셰퍼트',
     '차우차우',
     '닥스훈트',
     '보더콜리',
     '말라뮤트',
     '불독',
     '풍산개']
    #견종리스트에 최종적으로 이클립스에 보낼 리스트안에 추가
    for i in range(3):
        send_list[i] = dog_list[result_list[i]]
    for i in range(3):
        if send_list[i]== '골든 리트리버':
            send_list[i]='골든리트리버'
        elif send_list[i]== '미니핀':
            send_list[i]='미니어쳐핀셔'
        elif send_list[i]== '미니어쳐 핀셔':
            send_list[i]='미니어쳐핀셔'
    rank1 = send_list[0]
    rank2 = send_list[1]
    rank3 = send_list[2]


    url = 'http://localhost:8090/rebuild/test_result.jsp?rank1='+rank1+'&rank2='+rank2+'&rank3='+rank3
    return redirect(url)


if __name__ == '__main__':
    app.run(host='127.0.0.1', port='5000') 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jul/2021 13:29:52] "GET /dogmc?v1=S&v2=A&v3=I&v4=5&v5=5&v6=3&v7=2&v8=10u&v9=1&v10=3&v11=4 HTTP/1.1" 302 -


1순위 :  0.99025553
2순위 :  0.0015140552
3순위 :  0.0015044863
